In [1]:
import os

In [2]:
# Variable declaration
image_mode = "FLAIR" # FLAIR - T1w - T1wCE - T2w
patient_mode = "first" # first - biggest TODO: Explain 
input_dir = "../input/rsna-miccai-png/train" 
target_dir = "../input/rsna-miccai-png/test"

In [3]:
# Get path to the training images
input_img_paths_patients = sorted([
    os.path.join(input_dir, fname + "/" + image_mode) 
    for fname in os.listdir(input_dir)
    if os.path.exists(os.path.join(input_dir, fname + "/" + image_mode))])

input_img_paths = []
for patient in input_img_paths_patients:
    # TODO: Explain
    # TODO: Choosing first makes no sense. Because of the way images are taken. Center or bigger
    if patient_mode == "first":
        input_img_paths.append(os.path.join(patient, os.listdir(patient)[0]))


In [4]:
print(input_img_paths[0])

../input/rsna-miccai-png/train/00000/FLAIR/Image-79.png


In [5]:
os.listdir(input_img_paths_patients[0])

['Image-79.png',
 'Image-223.png',
 'Image-232.png',
 'Image-257.png',
 'Image-64.png',
 'Image-230.png',
 'Image-130.png',
 'Image-129.png',
 'Image-177.png',
 'Image-322.png',
 'Image-140.png',
 'Image-144.png',
 'Image-281.png',
 'Image-234.png',
 'Image-266.png',
 'Image-241.png',
 'Image-194.png',
 'Image-156.png',
 'Image-48.png',
 'Image-262.png',
 'Image-242.png',
 'Image-109.png',
 'Image-333.png',
 'Image-292.png',
 'Image-210.png',
 'Image-174.png',
 'Image-184.png',
 'Image-110.png',
 'Image-89.png',
 'Image-108.png',
 'Image-331.png',
 'Image-195.png',
 'Image-272.png',
 'Image-273.png',
 'Image-283.png',
 'Image-105.png',
 'Image-213.png',
 'Image-46.png',
 'Image-49.png',
 'Image-126.png',
 'Image-277.png',
 'Image-225.png',
 'Image-81.png',
 'Image-218.png',
 'Image-313.png',
 'Image-205.png',
 'Image-165.png',
 'Image-102.png',
 'Image-163.png',
 'Image-318.png',
 'Image-172.png',
 'Image-123.png',
 'Image-309.png',
 'Image-189.png',
 'Image-100.png',
 'Image-233.png',